In [ ]:
import numpy as np
import imutils
import cv2
import time
from google.colab.patches import cv2_imshow

In [ ]:
startX=0
startY=0
endX=0
endY=0

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo1.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
frame = imutils.resize(img, width=600)
(h, w) = frame.shape[:2]
modelFile = "/content/drive/MyDrive/Myphotos/model/res10_300x300_ssd_iter_140000_fp16 (1).caffemodel"
configFile = "/content/drive/MyDrive/Myphotos/model/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [ ]:
start=time.time()
blob=cv2.dnn.blobFromImage(frame,1.0, (300, 300), [104, 117, 123], False, False)

In [ ]:
net.setInput(blob)
detections=net.forward()
end=time.time()
print("Caffemodel:",format(end-start,'.2f'))
bboxes = []

Caffemodel: 3.41


In [ ]:
for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the prediction
	confidence = detections[0, 0, i, 2]
	#print(confidence)

		# filter out weak detections
	if confidence > 0.50:

			# compute the (x, y)-coordinates of the bounding box for
			# the face
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		cv2.rectangle(img, (startX , startY), (endX, endY),
			(0, 0, 255), 2)


In [ ]:
cv2.imwrite("output/result.jpg" ,img)
cv2_imshow(img)
if(cv2.waitKey(0) & 0xFF==27):
	cv2.destroyAllWindows()